In [ ]:
%reset
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import glob
import SimpleITK as sitk
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '../../dataloaders/')
import niftiio as nio

In [ ]:
# lets save them in a same way as mmwhs for the ease of modifying dataloader

# normalization: cut top 2% of histogram, then doing volume-wise normalization

IMG_BNAME="./normalized/image_*.nii.gz"
SEG_BNAME="./normalized/label_*.nii.gz"


In [ ]:
imgs = glob.glob(IMG_BNAME)
segs = glob.glob(SEG_BNAME)
imgs = [ fid for fid in sorted(imgs, key = lambda x: int(x.split("_")[-1].split(".nii.gz")[0])  ) ]
segs = [ fid for fid in sorted(segs, key = lambda x: int(x.split("_")[-1].split(".nii.gz")[0])  ) ]


In [ ]:
imgs

In [ ]:
lb = nio.read_nii_bysitk(segs[0])
lb.shape
1 in lb[..., 40]

In [ ]:
import json
classmap = {}
# LABEL_NAME = ["BGD", "SPLEEN", "KID_R", "KID_l", "GALLBLADDER", "ESOPHAGUS", "LIVER", "STOMACH", "AORTA", "IVC", "PS_VEIN", "PANCREAS", "AG_R", "AG_L"]
LABEL_NAME = ["BGD", "SPLEEN", "KID_R", "KID_L", "GALLBLADDER", "ESOPHAGUS", "LIVER", "STOMACH", "AORTA", "IVC",  "PANCREAS", "AG_R", "AG_L", "DUODENUM", "BLADDER", "PROSTATE"]

MIN_TP=1 # minimum number of true positive pixels in a slice

fid = f'./normalized/classmap_{MIN_TP}.json'
for _lb in LABEL_NAME:
    classmap[_lb] = {}
    for pid in range(len(segs)):
        classmap[_lb][str(pid)] = []

for pid, seg in enumerate(segs):
    lb_vol = nio.read_nii_bysitk(seg)
    n_slice = lb_vol.shape[0]
    for slc in range(n_slice):
        for cls in range(len(LABEL_NAME)):
            if cls in lb_vol[slc, ...]:
                if np.sum( lb_vol[slc, ...] ) >= MIN_TP:
                    classmap[LABEL_NAME[cls]][str(pid)].append(slc)
    print(f'pid {str(pid)} finished!')
    
with open(fid, 'w') as fopen:
    json.dump(classmap, fopen)
    fopen.close()  
    